# Web Driver Scraper 

Proyect Imports

In [266]:
from selenium import webdriver
import time

Create an instance of Chrome

In [194]:
#Instance in incognito mode
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument('--incognito')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver.exe', options=options)

In [2]:
#Instance in normal mode
#driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver.exe')

Pass the url to the chrome driver browse

In [15]:
#Flight form https://www.latam.com

page_path = 'https://www.latam.com/en_un/apps/personas/booking?fecha1_dia=07&fecha1_anomes=2021-02&from_city1=LIM&to_city1=MIA&ida_vuelta=ida_vuelta&fecha2_dia=18&fecha2_anomes=2021-02&from_city2=MIA&to_city2=LIM&cabina=Y&nadults=1&nchildren=0&ninfants=0&app=deal-finder#/'

In [173]:
driver.get(page_path)

Closes pop-up

In [174]:
driver.find_element_by_xpath('//button[@class="align-right secondary slidedown-button"]').click()

Extract info from the page

In [24]:
flights = driver.find_elements_by_xpath('//li[@class="flight"]')

In [25]:
#Display all the flights
len(flights)

9

In [26]:
# Select the second flight
flight = flights[2]
flight

<selenium.webdriver.remote.webelement.WebElement (session="e9f01b1485f54606172bccfcec1cd3bf", element="d1afb24d-985a-4f15-8198-4b5b502c1dd9")>

In [27]:
departure = flight.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
departure

'09:15'

In [28]:
arrival = flight.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
arrival

'05:35'

In [29]:
duration = flight.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')
duration

'PT20H20M'

In [30]:
stop_button = flight.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button')
stop_button.click()

In [32]:
flight_stops = driver.find_element_by_class_name('modal-body')
flight_stops

<selenium.webdriver.remote.webelement.WebElement (session="e9f01b1485f54606172bccfcec1cd3bf", element="dc456599-c823-4d3b-8bb8-99f3b5e3c07a")>

In [33]:
flight_scales = flight_stops.find_elements_by_class_name('sc-bsbRJL')
flight_scales

[<selenium.webdriver.remote.webelement.WebElement (session="e9f01b1485f54606172bccfcec1cd3bf", element="4b94d33e-6dd0-4c3c-9e03-75b87d88d20f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e9f01b1485f54606172bccfcec1cd3bf", element="fcbc7a30-b098-44fc-851d-b7b85cf101dd")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e9f01b1485f54606172bccfcec1cd3bf", element="d3e60b5b-307d-49a6-922d-d57db9dec64c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e9f01b1485f54606172bccfcec1cd3bf", element="38ac29c4-67d0-4ce8-ae65-8ac7532041e4")>]

In [60]:
flight_airports = []
[flight_airports.append(fs.find_element_by_class_name('sc-hXRMBi')) for fs in flight_scales]
for fa in flight_airports:
    print(fa.text)

J Chavez Intl.
Orlando International Airport
Orlando International Airport
WEST B HARTSFIELD
WEST B HARTSFIELD
Miami Intl.


In [46]:
flight_infos = []
for fs in flight_scales:
    flight_info = {}
    
    stop = fs.find_element_by_xpath('.//span[@class="sc-eTuwsz eumCTU"]')
    
    city = stop.find_element_by_xpath('.//abbr').text
    if city:
        flight_info['city'] = city
    else:
        flight_info['city'] = None
    
    hour = stop.find_element_by_xpath('.//time').text
    if hour:
        flight_info['hour'] = hour
    else:
        flight_info['hour'] = None
        
    airp = stop.find_element_by_xpath('.//span[@class="sc-hXRMBi gVvErD"]').text
    if airp:
        flight_info['airp'] = airp
    else:
        flight_info['airp'] = None
        
    flight_infos.append(flight_info)
print(flight_infos)
        

[{'city': 'LIM', 'hour': '09:15', 'airp': 'J Chavez Intl.'}, {'city': 'SCL', 'hour': '14:45', 'airp': 'A. Merino Benitez Intl.'}, {'city': 'SCL', 'hour': '23:10', 'airp': 'A. Merino Benitez Intl.'}, {'city': 'MIA', 'hour': '05:35+1', 'airp': 'Miami Intl.'}]


In [66]:
info_tramos = []
tramos = driver.find_elements_by_xpath('//div[@class="sc-cLQEGU hyoued"]')
for tramo in tramos:
    info_tramo = {}
    paradas = tramo.find_elements_by_xpath('.//span[@class="sc-eTuwsz eumCTU"]')
    
    i = 1
    for parada in paradas:
        info_paradas = {}
        out_city = parada.find_element_by_xpath('.//abbr').text
        if out_city:
            info_paradas['city'] = out_city
        else:
            info_paradas['city'] = None
        out_time = parada.find_element_by_xpath('.//time').get_attribute('datetime')
        if out_time:
            info_paradas['time'] = out_time
        else:
            info_paradas['time'] = None
        out_airp = parada.find_element_by_xpath('.//span[@class="sc-hXRMBi gVvErD"]').text
        if out_airp:
            info_paradas['airp'] = out_airp
        else:
            info_paradas['airp'] = None
        info_tramo['parada_'+str(i)] = info_paradas
        i += 1
    
    duration = tramo.find_element_by_xpath('.//div[@class="sc-bMVAic hShZwU"]//time').get_attribute('datetime')
    if duration:
        info_tramo['duration'] = duration
    else:
        info_tramo['duration'] = None
    
    divition = tramo.find_element_by_xpath('.//div[@class="sc-hMFtBS bGZqFm"]')
    
    flight = divition.find_element_by_xpath('.//b').text
    if flight:
        info_tramo['flight'] = flight
    else:
        info_tramo['flight'] = None
        
    plain = divition.find_element_by_xpath('.//span[@class="sc-gzOgki uTyOl"]').text
    if plain:
        info_tramo['plain'] = plain
    else:
        info_tramo['plain'] = None
        
    operated = divition.find_element_by_xpath('.//span[@class="sc-kfGgVZ lccoDs"]').text
    if operated:
        info_tramo['operated'] = operated
    else:
        info_tramo['operated'] = None
        
    info_tramos.append(info_tramo)

info_inter = {}
for i in range(len(tramos) - 1):
    inter = driver.find_element_by_xpath('//div[@class="sc-cLQEGU dnKRNG"]//div[@class="sc-hMFtBS cfwWiO"]')
    
    change_location = inter.find_element_by_xpath('.//span[@class="connection-label sc-hORach NXcGo"]').text
    if change_location:
        info_inter['change_location'] = change_location
    else:
        info_inter['change_location'] = None
    
    change_type = inter.find_element_by_xpath('.//span[@class="connection-changes sc-hORach NXcGo"]').text
    if change_type:
        info_inter['change_type'] = change_type
    else:
        info_inter['change_type'] = None
        
    change_time = inter.find_element_by_xpath('.//time').get_attribute('datetime')
    if change_time:
        info_inter['change_time'] = change_time
    else:
        info_inter['change_time'] = None
        
    change_kind = inter.find_element_by_xpath('.//strong[@class="sc-epnACN gCYixI"]').text
    if change_kind:
        info_inter['change_kind'] = change_kind
    else:
        info_inter['change_kind'] = None
        
    info_tramos.append(info_inter)
    
print(info_tramos)

[{'parada_1': {'city': 'LIM', 'time': '09:15', 'airp': 'J Chavez Intl.'}, 'parada_2': {'city': 'SCL', 'time': '14:45', 'airp': 'A. Merino Benitez Intl.'}, 'duration': '3:30', 'flight': 'LA2377', 'plain': 'Boeing 787-9', 'operated': 'Operated by LATAM Perú'}, {'parada_1': {'city': 'SCL', 'time': '23:10', 'airp': 'A. Merino Benitez Intl.'}, 'parada_2': {'city': 'MIA', 'time': '05:35', 'airp': 'Miami Intl.'}, 'duration': '8:25', 'flight': 'LA500', 'plain': 'Boeing 787-9', 'operated': 'Operated by LATAM Airlines'}, {'change_location': 'Connection at Santiago de Chile', 'change_type': '(change of aircraft)', 'change_time': '8:25', 'change_kind': 'Long stopover'}]


In [87]:
#close button
close_button = driver.find_element_by_xpath('//div[@class="modal-header sc-dnqmqq cGfTsx"]/button[@class="close"]')
close_button.click()

In [163]:
flights = driver.find_elements_by_xpath('//li[@class="flight"]')
flights[1].click()
#fl = flights[2]
#fl.click()

In [242]:
#fares
t1 = time.time()
fares_lists = []
delay = 5
flights = driver.find_elements_by_xpath('//li[@class="flight"]')
for fly in flights:
    fares_list = []
    print(fly)
    fly.click()
    ReactCollapse = fly.find_element_by_xpath('.//div[@class="ReactCollapse--content"]')
    if ReactCollapse:
        cabins = ReactCollapse.find_elements_by_xpath('.//ul[@class="cabin-selector"]/li')
        if cabins:
            for cabin in cabins:
                cabin.find_element_by_xpath('.//button').click()
                fares = ReactCollapse.find_elements_by_xpath('.//tfoot//td[contains(@class,"fare-")]')
                for fare in fares:
                    temp = {}
                    types = fare.find_element_by_xpath('.//label').get_attribute('for')
                    if types:
                        temp['type'] = types
                    else:
                        temp['type'] = None

                    price = fare.find_element_by_xpath('.//span[@class="price"]')

                    if price:
                        symbol = price.find_element_by_xpath('.//span[@class="currency-symbol"]')
                        if symbol:
                            temp['symbol'] = symbol.text
                        else:
                            temp['symbol'] = None

                        value = price.find_element_by_xpath('.//span[@class="value"]/span')
                        if value:
                            temp['value'] = value.text
                        else:
                            temp['value'] = None
                    else:
                        temp['symbol'] = None
                        temp['value'] = None
                    fares_list.append(temp)
        else:
            fares = ReactCollapse.find_elements_by_xpath('.//tfoot//td[contains(@class,"fare-")]')
            for fare in fares:
                temp = {}
                types = fare.find_element_by_xpath('.//label').get_attribute('for')
                if types:
                    temp['type'] = types
                else:
                    temp['type'] = None

                price = fare.find_element_by_xpath('.//span[@class="price"]')

                if price:
                    symbol = price.find_element_by_xpath('.//span[@class="currency-symbol"]')
                    if symbol:
                        temp['symbol'] = symbol.text
                    else:
                        temp['symbol'] = None

                    value = price.find_element_by_xpath('.//span[@class="value"]/span')
                    if value:
                        temp['value'] = value.text
                    else:
                        temp['value'] = None
                else:
                    temp['symbol'] = None
                    temp['value'] = None

                fares_list.append(temp)
    fares_lists.append(fares_list)
    time.sleep(delay)
t2 = time.time()
print(fares_lists)
t3 = t2 - t1
print(t3)

<selenium.webdriver.remote.webelement.WebElement (session="6d172f00d56f2d46162919f2f2fc195f", element="6d363810-9704-48af-aab0-9621482d8b26")>
<selenium.webdriver.remote.webelement.WebElement (session="6d172f00d56f2d46162919f2f2fc195f", element="9c08dc03-201d-4162-9518-5676a91e97ad")>
<selenium.webdriver.remote.webelement.WebElement (session="6d172f00d56f2d46162919f2f2fc195f", element="4c1726b0-fe04-4c9d-bf87-b9fa767a474f")>
<selenium.webdriver.remote.webelement.WebElement (session="6d172f00d56f2d46162919f2f2fc195f", element="4ebe9dd1-7705-4dae-9e73-cf7df5ef02ef")>
<selenium.webdriver.remote.webelement.WebElement (session="6d172f00d56f2d46162919f2f2fc195f", element="23233b98-cdfe-4d06-9021-b0da0c1181db")>
<selenium.webdriver.remote.webelement.WebElement (session="6d172f00d56f2d46162919f2f2fc195f", element="4ea67a95-f16c-4660-80f1-067197df4117")>
<selenium.webdriver.remote.webelement.WebElement (session="6d172f00d56f2d46162919f2f2fc195f", element="2423bf74-dd39-4926-93db-00a2f8840f5c")>

# construir Scraper


### Construir funciones

In [416]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ExConditions
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time
import sys

In [329]:
def ext_fare(fare, cabin_category=''):
    """Extract fare info
    Arguments:
        webelement fare: it provides the fare element
        string cabin_category: (optional) it provides the cabin categoty
    Return:
        dict nameless: return the fare info"""
    
    typ = fare.find_element_by_xpath('.//label').get_attribute('for') + (' ' + cabin_category if cabin_category != '' else '')
    symbol = fare.find_element_by_xpath('.//span[@class="price"]//span[@class="currency-symbol"]')
    value = fare.find_element_by_xpath('.//span[@class="price"]//span[@class="value"]/span')
    
    return {typ:{'symbol':symbol.text,'value':value.text}}

In [395]:
def get_fares(flight):
    """get_fares: Extract all fares info
    Arguments:
        webelement flight: it provides the flight element
    Return:
        list fares_list: return all flight fares"""
    
    fares_list = []
    ReactCollapse = flight.find_element_by_xpath('.//div[@class="ReactCollapse--content"]')
    
    if ReactCollapse:
        cabins = ReactCollapse.find_elements_by_xpath('.//ul[@class="cabin-selector"]/li')
        if cabins:
            for cabin in cabins:
                cabin_category = cabin.find_element_by_xpath('.//button/span').text
                cabin.find_element_by_xpath('.//button').click()
                fares = ReactCollapse.find_elements_by_xpath('.//tfoot//td[contains(@class,"fare-")]')
                
                for fare in fares:
                    fares_list.append(ext_fare(fare, cabin_category))
        else:
            fares = ReactCollapse.find_elements_by_xpath('.//tfoot//td[contains(@class,"fare-")]')
            for fare in fares:
                fares_list.append(ext_fare(fare))
                
    return fares_list

In [381]:
def get_time(flight):
    """get_time: time information
    Arguments:
        webelement flight: it provides the flight
    Return:
        dict nameless: return dict with time info"""
    
    departure = flight.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
    arrival = flight.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
    duration = flight.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')
    
    return {'departure':departure, 'arrival':arrival, 'duration':duration}

In [382]:
def get_stop(driver):
    """get_stop: 
    Arguments:
        webdriver driver: webdriver instance
    Return:
        dict nameless: return stops info"""
    
    info_tramos = []
    tramos = driver.find_elements_by_xpath('//div[@class="sc-cLQEGU hyoued"]')
    for tramo in tramos:
        info_tramo = {}
        paradas = tramo.find_elements_by_xpath('.//span[@class="sc-eTuwsz eumCTU"]')
        i = 1
        for parada in paradas:
            out_city = parada.find_element_by_xpath('.//abbr').text
            out_time = parada.find_element_by_xpath('.//time').get_attribute('datetime')
            out_airp = parada.find_element_by_xpath('.//span[@class="sc-hXRMBi gVvErD"]').text
            
            info_tramo['tramo_'+str(i)] = {'out_city': out_city, 'out_time': out_time, 'out_airp': out_airp}
            i += 1

        duration = tramo.find_element_by_xpath('.//div[@class="sc-bMVAic hShZwU"]//time').get_attribute('datetime')
        flight = tramo.find_element_by_xpath('.//div[@class="sc-hMFtBS bGZqFm"]//b').text
        plain = tramo.find_element_by_xpath('.//div[@class="sc-hMFtBS bGZqFm"]//span[@class="sc-gzOgki uTyOl"]').text
        operated = tramo.find_element_by_xpath('.//div[@class="sc-hMFtBS bGZqFm"]//span[@class="sc-kfGgVZ lccoDs"]').text

        info_tramos.append({'info': info_tramo, 'duration': duration, 'flight': flight, 'plain': plain, 'operated': operated})

    info_inter = []
    inters = driver.find_elements_by_xpath('//div[@class="sc-cLQEGU dnKRNG"]//div[@class="sc-hMFtBS cfwWiO"]')
    for inter in inters:
        change_location = inter.find_element_by_xpath('.//span[@class="connection-label sc-hORach NXcGo"]').text
        change_type = inter.find_element_by_xpath('.//span[@class="connection-changes sc-hORach NXcGo"]').text
        change_time = inter.find_element_by_xpath('.//time').get_attribute('datetime')
        try:
            change_kind = inter.find_element_by_xpath('.//strong[@class="sc-epnACN gCYixI"]').text
        except:
            change_kind = None
        
        info_inter.append({'change_location': change_location, 'change_type': change_type, 'change_time': change_time, 'change_kind': change_kind})
        
    return {'tramos': info_tramos, 'intercambios': info_inter}

In [332]:
def get_driver(page_path):
    """get_driver: create the webdriver instance
    Arguments:
        string page_path: it provides the webpage string
    Return:
        webdriver driver: return webdriver instance"""
    
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")
    options.add_argument('--incognito')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    driver = webdriver.Chrome(executable_path='./chromedriver/chromedriver.exe', options=options)
    driver.get(page_path)
    
    return driver

In [434]:
def obtener_info(driver):
    print('comienza el scraping...')

    #get flights
    flights = driver.find_elements_by_xpath('//li[@class="flight"]')

    print(f'esisten {len(flights)} vuelos en la pagina')

    fly_list = []
    for fly in flights:
        fly_id = fly.find_element_by_xpath('.//div[@class="flight-container"]').get_attribute('id')
        print('flight: ',fly_id)
        
        time_info = get_time(fly)
        fly.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button').click()
        stop_info = get_stop(driver)
        driver.find_element_by_xpath('//div[@class="modal-header sc-dnqmqq cGfTsx"]/button[@class="close"]').click()
        fly.click()
        WebDriverWait(driver, delay).until(ExConditions.presence_of_element_located((By.XPATH, '//div[@class="ReactCollapse--content"]')))
        fares_info = get_fares(fly)
        fly.find_element_by_xpath('.//div[@class="summary-container"]').click()
        fly_list.append({fly_id:{'time_info': time_info, 'fares_info': fares_info, 'stop_info': stop_info}})
    driver.close()
    return fly_list

In [431]:
def page_load():
    page_path = 'https://www.latam.com/en_un/apps/personas/booking?fecha1_dia=07&fecha1_anomes=2021-02&from_city1=LIM&to_city1=MIA&ida_vuelta=ida_vuelta&fecha2_dia=18&fecha2_anomes=2021-02&from_city2=MIA&to_city2=LIM&cabina=Y&nadults=1&nchildren=0&ninfants=0&app=deal-finder#/'
    driver = get_driver(page_path)
    delay = 10
    #get delay
    try:
        #closes pop-up
        WebDriverWait(driver, delay).until(ExConditions.presence_of_element_located((By.XPATH, '//button[@class="align-right secondary slidedown-button"]')))
        driver.find_element_by_xpath('//button[@class="align-right secondary slidedown-button"]').click()
        print('webpage has finished to load')
        return driver

    except TimeoutException:
        print('web page last too much to load')
        sys.exit()

In [438]:
def main():
    t1 = time.time()
    print('started time: ', t1)
    info = obtener_info(page_load())
    t2 = time.time()
    print('finished time: ', t2)
    print('total time: ', t2 - t1)

In [439]:
main()

started time:  1608086357.8409033
webpage has finished to load
comienza el scraping...
esisten 9 vuelos en la pagina
flight:  LA2460
flight:  LA2371LA500
flight:  LA2377LA500
flight:  LA2375LA500
flight:  LA2474LA6113LA6112
flight:  LA2474LA6119LA6112
flight:  LA2474DL1553DL1422
flight:  LA2474DL1107DL1422
flight:  LA2474DL2422DL1422
finished time:  1608086405.005815
total time:  47.16491174697876


In [171]:
driver.close()